In [1]:
import pymysql
conn = pymysql.connect(host='localhost',port=3306,db='lolgamequiz',user='root',passwd='root',charset='utf8')
def sele(sql):
    cur = conn.cursor()
    cur.execute(sql)
    result = list(cur.fetchall())
    result = [list(r) for r in result]
    conn.commit()
    cur.close()
    
    return result


In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [43]:
sql = 'select date, teamA, teamB, scoreA, scoreB, fullTeamA, fullTeamB from world2019'
data = sele(sql)
results = pd.DataFrame(data, columns=['date', 'teamA', 'teamB', 'scoreA', 'scoreB', 'fullTeamA', 'fullTeamB'])
print(results)

sql = 'select teamName, rank, region from rank2019'
data = sele(sql)
rank2019 = pd.DataFrame(data, columns=['teamName', 'rank', 'region'])
print(rank2019)

          date teamA teamB scoreA scoreB               fullTeamA  \
0   2019-10-02    CG   UOL      0      1           Clutch Gaming   
1   2019-10-02   DFM   SPY      0      1      DetonatioN FocusMe   
2   2019-10-02   MMM   UOL      1      0                 MAMMOTH   
3   2019-10-02   DFM   ISG      0      1      DetonatioN FocusMe   
4   2019-10-02   MMM    CG      0      1                 MAMMOTH   
5   2019-10-02   SPY   ISG      1      0                  Splyce   
6   2019-10-03   RYL   DWG      0      1             Royal Youth   
7   2019-10-03  MEGA   HKA      0      1                    MEGA   
8   2019-10-03   DWG   FLA      1      0           DAMWON Gaming   
9   2019-10-03  MEGA    LK      1      0                    MEGA   
10  2019-10-03   RYL   FLA      0      1             Royal Youth   
11  2019-10-03   HKA    LK      0      1      Hong Kong Attitude   
12  2019-10-04   UOL    CG      1      0    Unicorns Of Love.CIS   
13  2019-10-04   SPY   DFM      0      1        

In [44]:
# clean data
# cover data type
results = results.astype({'date':'datetime64', 'scoreA':'int64', 'scoreB':'int64'})
# fix team name
results['teamA'] = results['teamA'].apply(lambda x: x.upper())
results['teamB'] = results['teamB'].apply(lambda x: x.upper())

# sort
results = results.sort_values(by='date')
results = results.reset_index()
results = results.drop(['index'], axis=1)
results

####################
# rank2019
# region
rank2019['region'] = rank2019['region'].apply(lambda x:x.split('_')[-1])
# rank
rank2019 = rank2019.astype({'rank':'int64'})
#results = results.merge(rank2019, how='left', left_on='fullTeamA', right_on='teamName')
rank2019


,teamName,rank,region
0,FunPlus Phoenix,1,LPL
1,Royal Never Give Up,2,LPL
2,Invictus Gaming,3,LPL
3,Team Liquid,1,LCS
4,Top Esports,4,LPL
5,JD Gaming,5,LPL
6,Bilibili Gaming,6,LPL
7,Cloud9,2,LCS
8,EDward Gaming,7,LPL
9,LNG Esports,8,LPL


In [45]:
df = results['scoreA'].append(results['scoreB'])
df = pd.DataFrame(df)
df.columns = ['score']
gy = df['score'].groupby(df['score'])
print(gy.count())
print(results[results['scoreA']==results['scoreB']])

score
0    79
1    85
2     1
3    11
Name: score, dtype: int64
Empty DataFrame
Columns: [date, teamA, teamB, scoreA, scoreB, fullTeamA, fullTeamB]
Index: []


In [46]:
# won
results['Awon'] = False
results['Bwon'] = False
results['Awon'] = results['scoreA']>results['scoreB']
results['Bwon'] = results['scoreA']<results['scoreB']
results

,date,teamA,teamB,scoreA,scoreB,fullTeamA,fullTeamB,Awon,Bwon
0,2019-10-02,CG,UOL,0,1,Clutch Gaming,Unicorns Of Love.CIS,False,True
1,2019-10-02,DFM,SPY,0,1,DetonatioN FocusMe,Splyce,False,True
2,2019-10-02,MMM,UOL,1,0,MAMMOTH,Unicorns Of Love.CIS,True,False
3,2019-10-02,DFM,ISG,0,1,DetonatioN FocusMe,Isurus,False,True
4,2019-10-02,MMM,CG,0,1,MAMMOTH,Clutch Gaming,False,True
5,2019-10-02,SPY,ISG,1,0,Splyce,Isurus,True,False
6,2019-10-03,RYL,DWG,0,1,Royal Youth,DAMWON Gaming,False,True
7,2019-10-03,MEGA,HKA,0,1,MEGA,Hong Kong Attitude,False,True
8,2019-10-03,DWG,FLA,1,0,DAMWON Gaming,Flamengo eSports,True,False
9,2019-10-03,MEGA,LK,1,0,MEGA,Lowkey Esports.Vietnam,True,False


In [50]:

############################################
# region
results['regionA'] = None
results['regionB'] = None
results['AisLCK'] = False
results['AisLCS'] = False
results['AisLPL'] = False
results['AisLMS'] = False
results['BisLCK'] = False
results['BisLCS'] = False
results['BisLPL'] = False
results['BisLMS'] = False
for index, row in results.iterrows():
    teamA = row['fullTeamA']
    teamB = row['fullTeamB']
    regionA = rank2019[rank2019['teamName']==teamA]
    if not regionA.empty:
        regionA = regionA['region'].values[0]
        results.loc[index,'regionA'] = regionA
        if regionA == 'LCK':
            results.loc[index,'AisLCK'] = True
        elif regionA == 'LCS':
            results.loc[index,'AisLCS'] = True
        elif regionA == 'LPL':
            results.loc[index,'AisLPL'] = True
        elif regionA == 'LMS':
            results.loc[index,'AisLMS'] = True
    
    regionB = rank2019[rank2019['teamName']==teamB]
    if not regionB.empty:
        regionB = regionB['region'].values[0]
        results.loc[index,'regionB'] = regionB
        if regionB == 'LCK':
            results.loc[index,'BisLCK'] = True
        elif regionB == 'LCS':
            results.loc[index,'BisLCS'] = True
        elif regionB == 'LPL':
            results.loc[index,'BisLPL'] = True
        elif regionB == 'LMS':
            results.loc[index,'BisLMS'] = True

############################################
# rank
results['rankA'] = 5
results['rankB'] = 5
results['Ahigh'] = False
results['Bhigh'] = False
for index, row in results.iterrows():
    teamA = row['fullTeamA']
    teamB = row['fullTeamB']
    rankA = rank2019[rank2019['teamName']==teamA]
    if not rankA.empty:
        rankA = rankA['rank'].values[0]
    else:
        rankA = 5

    rankB = rank2019[rank2019['teamName']==teamB]
    if not rankB.empty:
        rankB = rankB['rank'].values[0]
    else:
        rankB = 5
    
    if rankA < rankB:
        results.loc[index,'Ahigh'] = True
    if rankB < rankA:
        results.loc[index,'Bhigh'] = True
        
    results.loc[index,'rankA'] = rankA
    results.loc[index,'rankB'] = rankB
    

##################################
# last won
results['Alastwon'] = False
results['Blastwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        won_last[teamA] = row['Awon']
        won_last[teamB] = row['Bwon']
        continue

    results.loc[index, 'Alastwon'] = won_last[teamA]
    results.loc[index, 'Blastwon'] = won_last[teamB]
    won_last[teamA] = row['Awon']
    won_last[teamB] = row['Bwon']

##################################
# last who won
results['lastAwon'] = False
results['lastBwon'] = False
won_last = defaultdict(bool)
for index, row in results.iterrows():
    teamA = row['teamA']
    teamB = row['teamB']
    if index == 0:
        continue
    for index1, row1 in results[:index].iterrows():
        if row1['teamA'] == teamA and row1['teamB'] == teamB:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Awon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Bwon']
        elif row1['teamA'] == teamB and row1['teamB'] == teamA:
            results.loc[index, 'lastAwon'] = results.loc[index1, 'Bwon']
            results.loc[index, 'lastBwon'] = results.loc[index1, 'Awon']

##################################
# BO
def bo(score, win_num):
    max_score = max(score['scoreA'], score['scoreB'])
    if max_score == win_num:
        return True
    else:
        return False
        
results['isBo1'] = results[['scoreA','scoreB']].apply(bo, args=(1,), axis=1)
results['isBo3'] = results[['scoreA','scoreB']].apply(bo, args=(2,), axis=1)
results['isBo5'] = results[['scoreA','scoreB']].apply(bo, args=(3,), axis=1)

results



,date,teamA,teamB,scoreA,scoreB,fullTeamA,fullTeamB,Awon,Bwon,regionA,...,rankB,Ahigh,Bhigh,Alastwon,Blastwon,lastAwon,lastBwon,isBo1,isBo3,isBo5
0,2019-10-02,CG,UOL,0,1,Clutch Gaming,Unicorns Of Love.CIS,False,True,LCS,...,5,True,False,False,False,False,False,True,False,False
1,2019-10-02,DFM,SPY,0,1,DetonatioN FocusMe,Splyce,False,True,None,...,3,False,True,False,False,False,False,True,False,False
2,2019-10-02,MMM,UOL,1,0,MAMMOTH,Unicorns Of Love.CIS,True,False,None,...,5,False,False,False,True,False,False,True,False,False
3,2019-10-02,DFM,ISG,0,1,DetonatioN FocusMe,Isurus,False,True,None,...,5,False,False,False,False,False,False,True,False,False
4,2019-10-02,MMM,CG,0,1,MAMMOTH,Clutch Gaming,False,True,None,...,3,False,True,True,False,False,False,True,False,False
5,2019-10-02,SPY,ISG,1,0,Splyce,Isurus,True,False,LEC,...,5,True,False,True,True,False,False,True,False,False
6,2019-10-03,RYL,DWG,0,1,Royal Youth,DAMWON Gaming,False,True,None,...,3,False,True,False,False,False,False,True,False,False
7,2019-10-03,MEGA,HKA,0,1,MEGA,Hong Kong Attitude,False,True,None,...,3,False,True,False,False,False,False,True,False,False
8,2019-10-03,DWG,FLA,1,0,DAMWON Gaming,Flamengo eSports,True,False,LCK,...,5,True,False,True,False,False,False,True,False,False
9,2019-10-03,MEGA,LK,1,0,MEGA,Lowkey Esports.Vietnam,True,False,None,...,5,False,False,False,False,False,False,True,False,False


In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


In [58]:
X_previouswins = results[['AisLCK','AisLCS','AisLPL','AisLMS','BisLCK','BisLCS','BisLPL','BisLMS','Ahigh','Bhigh']].values
X_previouswins = results[['AisLCK','AisLCS','AisLPL','AisLMS','BisLCK','BisLCS','BisLPL','BisLMS','rankA','rankB']].values
X_previouswins = results[['AisLCK','AisLCS','AisLPL','AisLMS','BisLCK','BisLCS','BisLPL','BisLMS','rankA','rankB','Alastwon','Blastwon']].values
X_previouswins = results[['AisLCK','AisLCS','AisLPL','AisLMS','BisLCK','BisLCS','BisLPL','BisLMS','rankA','rankB','Alastwon','Blastwon','isBo1','isBo3','isBo5']].values
#X_previouswins = results['Alastwon','Blastwon',['AisLCK','AisLCS','AisLPL','AisLMS','BisLCK','BisLCS','BisLPL','BisLMS','rankA','rankB','Alastwon','Blastwon','lastAwon','lastBwon']].values

y_true = results[['Awon','Bwon']].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print('score: {:.2f}%'.format(np.mean(scores)*100))


score: 67.13%
